# Fix and Implement Supabase Edge Function for RSS Ingestion

This notebook will guide you through identifying, fixing, and implementing the Supabase Edge Function required to fetch and ingest RSS feeds into your `news_articles` table. It will also show how to test and verify the solution.

## 1. Identify and Analyze the Problem

- The frontend fails to call the Supabase Edge Function (`fetchFeeds`) for RSS ingestion.
- Possible causes: function not deployed, wrong function name, CORS, or code issues.
- Required: A working Edge Function that fetches all RSS feeds, parses them, and upserts articles into `news_articles`.
- We will implement, deploy, and test this function.

## 2. Import Required Libraries

We will use Deno, the Supabase JS client, and a simple XML parser for RSS feeds. These are available in the Edge Function runtime.

In [ ]:
// Edge Function: import libraries
import { serve } from "https://deno.land/std@0.168.0/http/server.ts";
import { createClient } from "https://esm.sh/@supabase/supabase-js@2.39.7";


## 3. Load and Inspect the Source Code

Below is the core logic for the Edge Function. It fetches all RSS feeds, parses them, and upserts articles into the database. We'll inspect and improve it as needed.

In [ ]:
// Edge Function: fetchFeeds (initial version)
serve(async (req) => {
  const supabase = createClient(
    Deno.env.get("SUPABASE_URL")!,
    Deno.env.get("SUPABASE_SERVICE_ROLE_KEY")!
  );

  // 1. Get all feeds
  const { data: feeds, error: feedsError } = await supabase
    .from("rss_feeds")
    .select("*");
  if (feedsError) return new Response(JSON.stringify({ error: feedsError.message }), { status: 500 });

  // 2. For each feed, fetch and parse RSS
  for (const feed of feeds) {
    try {
      const res = await fetch(feed.url);
      const xml = await res.text();
      // Parse XML (simple regex for <item> blocks)
      const items = Array.from(xml.matchAll(/<item>([\s\S]*?)<\/item>/g));
      for (const item of items) {
        const title = item[1].match(/<title>(.*?)<\/title>/)?.[1] ?? "";
        const link = item[1].match(/<link>(.*?)<\/link>/)?.[1] ?? "";
        const pubDate = item[1].match(/<pubDate>(.*?)<\/pubDate>/)?.[1] ?? "";
        if (!title || !link) continue;
        await supabase.from("news_articles").upsert({
          title,
          link,
          source: feed.source,
          category: feed.category,
          published: pubDate ? new Date(pubDate).toISOString() : new Date().toISOString(),
        }, { onConflict: "link" });
      }
    } catch (e) {
      // Log and continue
      console.error(`Failed to fetch or parse feed: ${feed.url}`, e);
    }
  }

  return new Response(JSON.stringify({ status: "ok" }), { status: 200 });
});

## 4. Apply Fixes to the Code

- Use a more robust XML parser if needed (for production, use a library).
- Ensure all required fields are extracted (title, link, published, source, category).
- Add error handling and logging for debugging.
- Ensure CORS headers are set if needed for local testing.

In [ ]:
// Improved: Add CORS, better error handling, and log output
serve(async (req) => {
  const supabase = createClient(
    Deno.env.get("SUPABASE_URL")!,
    Deno.env.get("SUPABASE_SERVICE_ROLE_KEY")!
  );
  const logs: string[] = [];
  const { data: feeds, error: feedsError } = await supabase
    .from("rss_feeds")
    .select("*");
  if (feedsError) return new Response(JSON.stringify({ error: feedsError.message }), { status: 500, headers: { 'Access-Control-Allow-Origin': '*' } });

  for (const feed of feeds) {
    try {
      const res = await fetch(feed.url);
      const xml = await res.text();
      const items = Array.from(xml.matchAll(/<item>([\s\S]*?)<\/item>/g));
      for (const item of items) {
        const title = item[1].match(/<title>(.*?)<\/title>/)?.[1] ?? "";
        const link = item[1].match(/<link>(.*?)<\/link>/)?.[1] ?? "";
        const pubDate = item[1].match(/<pubDate>(.*?)<\/pubDate>/)?.[1] ?? "";
        if (!title || !link) continue;
        await supabase.from("news_articles").upsert({
          title,
          link,
          source: feed.source,
          category: feed.category,
          published: pubDate ? new Date(pubDate).toISOString() : new Date().toISOString(),
        }, { onConflict: "link" });
        logs.push(`Upserted: ${title}`);
      }
    } catch (e) {
      logs.push(`Failed to fetch or parse feed: ${feed.url} - ${e}`);
    }
  }
  return new Response(JSON.stringify({ status: "ok", logs }), { status: 200, headers: { 'Access-Control-Allow-Origin': '*' } });
});

## 5. Implement Missing Functionality

- If you want to parse more fields (image, summary, etc.), add more regex or use a Deno XML parser.
- For production, use a robust XML parser and handle edge cases for different RSS formats.

## 6. Run and Test the Updated Code

- Deploy the function with the Supabase CLI:
  ```sh
  supabase functions deploy fetchFeeds
  ```
- Test from the Supabase dashboard or your frontend.
- Check the `news_articles` table for new articles after running.

## 7. Display Output and Verify Results

- If successful, the function will return `{ status: "ok", logs: [...] }`.
- The `news_articles` table should be populated with articles from all RSS feeds.
- If you see errors, check the logs in the function response and Supabase dashboard.